Test problem from lecture 3 of ME 522

Import the modules needed to solve the linear problem

In [1]:
from mpi4py import MPI
from petsc4py.PETSc import ScalarType
import numpy as np
import ufl
from dolfinx import fem, io, mesh, plot
from dolfinx.fem.petsc import LinearProblem
from ufl import ds, dx, grad, inner

Create the mesh and function space

In [2]:
msh = mesh.create_interval(comm=MPI.COMM_WORLD, points=((0.0),(1.0)), nx=20) # Create mesh
V = fem.functionspace(msh, ("Lagrange", 1)) # Create linear function space

Define the Boundary Conditions

In [3]:
# Lower Boundary Condition
facets_lower = mesh.locate_entities_boundary( # Find the facets of the lower boundary, there should only be 1 becuase this is a 1D problem
    msh,
    dim=(msh.topology.dim - 1),
    marker=lambda x: np.isclose(x[0], 0.0),
)
# Find the DOFs
dofs = fem.locate_dofs_topological(V=V, entity_dim=0, entities=facets_lower)
# Apply the lower BC
bc_L = fem.dirichletbc(value=ScalarType(0), dofs=dofs, V=V)

In [4]:
# Upper Boundary Condition
facets_upper = mesh.locate_entities_boundary( # Find the facets of the upper boundary, there should only be 1 becuase this is a 1D problem
    msh,
    dim=(msh.topology.dim - 1),
    marker=lambda x: np.isclose(x[0], 1.0),
)
# Find the DOFs
dofs = fem.locate_dofs_topological(V=V, entity_dim=0, entities=facets_upper)
# Apply the lower BC
bc_U = fem.dirichletbc(value=ScalarType(-0.4545), dofs=dofs, V=V)

# Combine boundary conditions
bcs = [bc_L, bc_U]

Define the variational problem

In [5]:
u = ufl.TrialFunction(V)
v = ufl.TestFunction(V)

a = ufl.dot(ufl.grad(u), ufl.grad(v)) * ufl.dx + 4 * u * v * ufl.dx
L = - 3 * v * ufl.dx

Solve the linear problem

In [6]:
from dolfinx.fem.petsc import LinearProblem
problem = LinearProblem(a, L, bcs=bcs, petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
uh = problem.solve()

In [7]:
xyz = V.tabulate_dof_coordinates()
x = xyz[:,0]
print(x)

[0.   0.05 0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65
 0.7  0.75 0.8  0.85 0.9  0.95 1.  ]


In [8]:
sol = np.array((uh.vector))

In [9]:
np.savetxt('FE_Practice_1_Mesh.csv', x, delimiter=',')
np.savetxt('FE_Practice_1_Sol.csv', sol, delimiter=',')